# Sentence Retrieval

In [129]:
import json,math,nltk

train_file = open("train.json",'r')
dev_file = open("dev.json",'r')
test_file=open("test.json",'r')
train = json.loads(train_file.read())
dev = json.loads(dev_file.read())
test = json.loads(test_file.read())
# train[0]['sentences']
# train[0]['qa']

In [130]:
from nltk.corpus import stopwords
# may be stem?

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
stopword =  stopwords.words()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def doc_word_dict(doc):
    word_dict = set()
    for sent in doc['sentences']:
        for word in  word_tokenizer.tokenize(sent):
            word = lemmatize(word.lower())
            if word not in stopword:
                word_dict.add(word)
    return word_dict

In [131]:
def get_BOW(sent):
    term_dict={}
    for word in word_tokenizer.tokenize(sent):
        word = lemmatize(word.lower())
        if word not in stopword:
            term_dict[word]=term_dict.setdefault(word,0)+1
    return term_dict

def cal_BOW(doc):
    doc_term_matrix = [] 
    for sent in doc['sentences']:
        temp = get_BOW(sent)
        doc_term_matrix.append(temp)
    return doc_term_matrix

In [132]:
def get_best_doc_num(query):
    query =  transformer.transform(vectorizer.transform(get_BOW(query)))
    f = lambda x: cos_distance(query.toarray(),term_matrix[x].toarray())
    best_doc = min(range(term_matrix.shape[0]),key=f)
    return best_doc

In [133]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance

vectorizer = DictVectorizer()
transformer = TfidfTransformer(smooth_idf=False,norm=None)

match_sent= []
count = 0
for test_doc in test:
    print count
    count += 1
    doc_match_sent = []
    term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(test_doc)))
    for qa in test_doc['qa']:
        doc_match_sent.append(get_best_doc_num(qa['question']))
    match_sent.append(doc_match_sent)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [134]:
len(match_sent)

42

# Entity Extraction

In [135]:
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz',
               '/usr/share/stanford-ner/stanford-ner.jar') 

test_tag = []
for i in range(len(match_sent)):
    test_sent_tag = []
    for j in match_sent[i]:
        test_sent_tag.append(word_tokenizer.tokenize(test[i]['sentences'][j]))
    test_sent_tag = st.tag_sents(test_sent_tag)
    test_tag.append(test_sent_tag)

In [136]:
len(test_tag[0])

287

In [137]:
def tune_other(tag_list):
    for i in range(len(tag_list)):
        for j in range(len(tag_list[i])):
            for k in range(len(tag_list[i][j])):
                term,tag = tag_list[i][j][k]
                if term!='' and (tag == "ORGANIZATION"  or (len(term)>0 and (term,tag)!=tag_list[i][j][0] and tag == 'O' and term[0].isupper())):
                    tag_list[i][j][k] = (term,"OTHER")

tune_other(test_tag)

In [138]:
def combine_entity(tag_list):
    for k in range(len(tag_list)):
        for i in range(len(tag_list[k])):
            j = 0
            while j < len(tag_list[k][i])-2:
                term,tag = tag_list[k][i][j]
                term_n,tag_n = tag_list[k][i][j+1]
                if tag == tag_n and tag != "O":
                    temp =  (term + " " + term_n,tag)
                    tag_list[k][i][j] = temp
                    del tag_list[k][i][j+1]
                j += 1

combine_entity(test_tag)

In [139]:
test_list = []
for k in test_tag:
    test_doc_list = []
    for i in k:
        test_ = []
        for term,tag in i:
            if term != '':
                test_.append(term)
        test_doc_list.append(test_)
    test_list.append(test_doc_list)

In [140]:
len(test_list[0])

287

In [141]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

tags = ["PERSON","LOCATION","NUMBER","OTHER"]
entity_pool = []
for k in test_tag:
    entity_doc_pool = []
    for i in k:
        sent_tag_dict = dict.fromkeys(tags,[])
        for j in range(len(i)):
            term,tag = i[j]
            if tag == "PERSON" or tag == "LOCATION" or tag == "OTHER":
                sent_tag_dict[tag] = sent_tag_dict[tag]+ [term]
            elif tag == "DATE" or tag == "TIME" or tag == "PERCENT" or hasNumbers(term):
                sent_tag_dict["NUMBER"] = sent_tag_dict["NUMBER"]+ [term]
        entity_doc_pool.append(sent_tag_dict)
    entity_pool.append(entity_doc_pool)

In [142]:
entity_pool[0]

[{'LOCATION': [], 'NUMBER': [u'350'], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [u'350'], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [],
  'NUMBER': [],
  'OTHER': [u'Christian', u'Holy Land', u'Ottoman Empire'],
  'PERSON': []},
 {'LOCATION': [u'Russia'],
  'NUMBER': [],
  'OTHER': [u'French', u'Catholics', u'Eastern Orthodox', u'Christians'],
  'PERSON': []},
 {'LOCATION': [u'Russia'],
  'NUMBER': [],
  'OTHER': [u'French', u'Catholics', u'Eastern Orthodox', u'Christians'],
  'PERSON': []},
 {'LOCATION': [u'Russia'],
  'NUMBER': [],
  'OTHER': [u'I'],
  'PERSON': [u'Nicholas', u'Napoleon III']},
 {'LOCATION': [],
  'NUMBER': [],
  'OTHER': [u'Orthodox', u'Empire'],
  'PERSON': []},
 {'LOCATION': [u'Britain'],
  'NUMBER': [],
  'OTHER': [],
  'PERSON': [u'Nicholas']},
 {'LOCATION': [],
  'NUMBER': [],
  'OTHER': [u'Ottomans'],
  'PERSON': [u'Nicholas']},
 {'LOCATION': [u'France', u'Britain', u'Russia'],
  'NUMBER': [u'October 1853'],
  'OTHER': [u'Ottomans'],
  'PERSON': 

# Answer Ranking

In [143]:
import operator
import nltk

# A simple rule-based question type classifier based on key words 
def get_question_type(question):
    # TODO: HAND-CODED, NEED TO BE REFINED!!
    # TODO: need to low-case to compare?
    
    type_rules = {'PERSON':["Who", "Whose", "Whom"],
                  'LOCATION':["Where"],
                  'NUMBER':["When", "few", "little", "much", "many",
                            "young", "old", "long", "year", "years"]
                 }
    q_type = None
    for question_type, key_words in type_rules.items():
        for key_word in key_words:
            if key_word in question:
                q_type = question_type
                break
        if q_type == None:
            q_type = 'OTHER'
    return q_type

In [144]:
from nltk import word_tokenize

# among entities of the same type, the prefered entity should be 
# the one which is closer in the sentence to a open-class word
# from the question.
# ----> nouns, verbs, adjectives, and adverbs.
def get_preferred_entity(entity_list, sentence, question):
    preferred_entity = None
    sentence_text = sentence
    question_text = word_tokenize(question)
    sentence_tag = nltk.pos_tag(sentence_text,tagset='universal')
    question_tag = nltk.pos_tag(question_text,tagset='universal')
    
    # initialize a list for comparing, and set all elements as 0
    is_open_word = [0] * len(sentence_text)
    # find an open word in the question
    for word, tag in question_tag:
        if tag in ['ADJ','NOUN','VERB','ADV']:
            # if the open word appears in the sentence, then mark as 1
            for i in range(len(sentence_text)):
                if sentence_text[i] == word:
                    is_open_word[i] = 1
#     print is_open_word
    
    # find the closest distance to an open-class word for an entity
    def get_distance(entity):
        # get the position of entity, and find the open class word 
        # from the nearest at both sides
        distance = None
        position = sentence_text.index(entity)
        for i in range(1, len(sentence_text)):
            if position - i >= 0:
                if is_open_word[position - i] == 1:  # find an open-class word on the left
                    distance = i
                    break
                elif position + i < len(is_open_word):  # find an open-class word on the right
                    if is_open_word[position + i] == 1:
                        distance = i
                        break
                else:
                    distance = len(sentence_text) + 1  # didn't find open-class words
        return distance
    
    # get distance for each entity and choose the best one
    all_distance = []
    for entity in entity_list:
        all_distance.append(get_distance(entity))
        preferred_entity = entity_list[all_distance.index(min(all_distance))]

    return preferred_entity
    

In [145]:
def get_ranked_ans(entities_dic, question, sentence):
    
    # identify if the entity set is empty. If True, return nothing
    is_empty = True
    for values in entities_dic.values():
        if len(values) != 0:
            is_empty = False
            
    if is_empty == False:
        q_type = get_question_type(question)
        tmp_rank = {}
        for ent_type,entities in entities_dic.items():
            # answers whose content words all appear in the question should be ranked lowest.
            for entity in entities:
                if entity in question:
                    tmp_rank[entity] = tmp_rank.setdefault(entity,0) - 1
            # Answers which match the question type should be ranked higher than those that don't
            if ent_type == q_type:
                for entity in entities:
                    tmp_rank[entity] = tmp_rank.setdefault(entity,0) + 1
                ######## TODO: Apply this to all types?
                    # entity closer in the sentence to a closed-class word should be preferred
                    tmp_rank[get_preferred_entity(entities, sentence, question)] = tmp_rank.setdefault(entity,0) + 1
        # sort and choose the best answer
        sorted_ans = sorted(tmp_rank.items(), key=operator.itemgetter(1), reverse=True)
        #####TODO: bug here. list out of index??? why?
        if len(sorted_ans) != 0:
            best_ans = sorted_ans[0][0]
        else:
            best_ans = ''
        return best_ans
    else:
        return ''

In [146]:
print train[0]["qa"][3]['answer_sentence']
print correct_rate[0][3]
print entity_pool[3]
print train[0]["qa"][3]['question']
print train[0]["sentences"][correct_rate[0][96]]
print train_list[3]

print 
print train[0]["qa"][12]['answer'] == get_ranked_ans(entity_pool[12], train[0]["qa"][12]['question'], train_list[12])

1
30
[{'PERSON': [], 'OTHER': [u'Old Avestan', u'Iranian'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Iranian', u'Iranic', u'Indo', u'Iranian', u'Indo', u'European'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Iranian', u'Iranic', u'Indo', u'Iranian', u'Indo', u'European'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Persian'], 'LOCATION': [], 'NUMBER': [u'second half', u'of the', u'8th century']}, {'PERSON': [], 'OTHER': [u'Iranian', u'Persian', u'Sassanid Iran', u'), Parthian', u'Bactrian'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Middle Iranian', u'Iranian'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Old Persian', u'Avestan', u'Old Iranian', u'Middle Iranian', u'Old Iranian', u'Old'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Persian', u'Sanskrit', u'Arya'], 'LOCATION': [], 'NUMBER': []}, {'PERSON': [], 'OTHER': [u'Iranian', u'Christian Lassen'], 'LOCATION': [], 'NUMBER': [u'1836']}, {'PER

AttributeError: 'list' object has no attribute 'values'

In [157]:
count = 1
with open("result.txt",'w') as output_file:
    for i in range(len(match_sent)):
        for j in range(len(match_sent[i])):
    #         cor_answer = test[i]["qa"][j]['answer']
            result = get_ranked_ans(entity_pool[i][j], test[i]["qa"][j]['question'], test_list[i][j])
            q_id = test[i]["qa"][j]['id']
            count += 1
            string = str(q_id)+","+result.encode('utf-8')+"\n"
            output_file.write(string)
    #         if result == cor_answer:
    #             sum += 1
    # print sum

1 , 350
2 , 350
3 , Holy Land
4 , 
5 , Christians
6 , I
7 , Orthodox
8 , Nicholas
9 , Nicholas
10 , Ottomans
11 , 
12 , Russian
13 , 
14 , 
15 , Ottoman
16 , Russians
17 , 201
18 , Russians
19 , Omar Pasha
20 , Russians
21 , September
22 , 
23 , 
24 , 30
25 , Christian
26 , Crimean War
27 , 
28 , Preface
29 , April 1854
30 , Janissary
31 , June 1826
32 , Franco
33 , 1830
34 , 1830
35 , Muhammad Ali
36 , 
37 , Russian
38 , 10
39 , Ali
40 , Ottoman
41 , June 24
42 , Muhammad Ali
43 , 1840
44 , Europe
45 , Treaty
46 , 
47 , Nicholas
48 , 
49 , 200
50 , 
51 , 
52 , Ottoman
53 , Russian
54 , 
55 , 
56 , 
57 , October 1853
58 , Ottomans
59 , Marquis Charles
60 , The
61 , La
62 , Black Sea
63 , Black Sea
64 , 
65 , Nativity
66 , Abdülmecid I
67 , British
68 , 350
69 , Prince Menshikov
70 , 105
71 , Empire
72 , British
73 , Ottoman Empire
74 , Stratford Canning
75 , Stratford
76 , 
77 , Menshikov
78 , Menshikov
79 , Danubian Principalities
80 , 000
81 , Russian
82 , Moldavia
83 , Russia
84 , O

648 , Contest
649 , Eurovision Song
650 , Greece
651 , Western
652 , 776
653 , BC
654 , Olympics
655 , 30
656 , 12th
657 , Greek
658 , 12th
659 , League
660 , League
661 , Greek
662 , Europe
663 , 2005
664 , Championship
665 , A1
666 , 2011
667 , World Aquatics
668 , World Aquatics
669 , World Aquatics
670 , Championships
671 , 6th
672 , Greek
673 , Greek
674 , Europe
675 , Greek
676 , Greek
677 , Greek
678 , Zeus
679 , Zeus
680 , Sunday
681 , Sunday
682 , 
683 , Greek
684 , 2012
685 , 53 %
686 , 2013
687 , April 2014
688 , Greece
689 , Alexander III
690 , Alexander III
691 , 492
692 , 330
693 , 334
694 , 
695 , 
696 , 
697 , 
698 , 
699 , 
700 , 
701 , 
702 , 
703 , 
704 , 
705 , 
706 , Lempel
707 , Gzip
708 , LZR
709 , LZ
710 , CAB
711 , 
712 , 
713 , 
714 , 
715 , 
716 , 
717 , CD
718 , 
719 , 
720 , CD
721 , CD
722 , 
723 , LPC
724 , 
725 , 
726 , 
727 , 
728 , 
729 , 
730 , fidelity
731 , Vorbis
732 , 
733 , 50
734 , 
735 , DVDs
736 , 
737 , 
738 , 
739 , 
740 , 
741 , 
742 , LPC


1354 , Charter
1355 , Charter
1356 , 
1357 , Charter
1358 , United Nations
1359 , United Nations
1360 , United Nations
1361 , Legal Affairs
1362 , United Nations
1363 , UN
1364 , UN
1365 , UN
1366 , States Federal
1367 , UN
1368 , Brazilian
1369 , Congress
1370 , the Federal
1371 , the Federal
1372 , VIII
1373 , Federal Supreme
1374 , 2008
1375 , Congress
1376 , 
1377 , Court
1378 , Brazilian
1379 , Brazilian
1380 , Brazilian
1381 , Brazilian
1382 , 
1383 , 
1384 , Senators
1385 , President
1386 , House
1387 , United States
1388 , House
1389 , State
1390 , President
1391 , Treaty
1392 , Head Money
1393 , The Supreme
1394 , Covert
1395 , Acts
1396 , Court
1397 , Union List
1398 , Parliament
1399 , Parliament
1400 , Parliament
1401 , Parliament
1402 , European
1403 , European
1404 , European
1405 , European
1406 , European
1407 , European
1408 , European
1409 , Qing Dynasty
1410 , European
1411 , Qing Dynasty
1412 , 
1413 , Australians
1414 , Australians
1415 , Australians
1416 , UN
1417

1900 , Development
1901 , Institute of
1902 , Russian
1903 , Russian
1904 , Russian
1905 , 
1906 , 
1907 , 
1908 , The
1909 , 
1910 , 
1911 , 
1912 , 
1913 , 1000
1914 , Rus
1915 , Rus
1916 , Rus
1917 , Kievan Rus
1918 , Western Europe
1919 , Russian
1920 , Russian
1921 , Leo Tolstoy
1922 , Russian
1923 , Russian
1924 , Russian
1925 , Russian
1926 , Pushkin
1927 , 20th century
1928 , 1990
1929 , Russian
1930 , Demoskop Weekly
1931 , 21st
1932 , 2012
1933 , A
1934 , 
1935 , Switzerland
1936 , Bern
1937 , 36
1938 , Bern
1939 , Bern
1940 , 
1941 , Bern
1942 , Bern
1943 , Alemannic Swiss
1944 , Alemannic Swiss
1945 , Alemannic Swiss
1946 , Alemannic Swiss
1947 , 1983
1948 , Bern
1949 , Bern
1950 , 2010
1951 , Bern
1952 , Bern
1953 , 1983
1954 , Berchtold
1955 , V
1956 , Bärengraben
1957 , Bern
1958 , Celtic
1959 , Bern
1960 , Bärengraben
1961 , Bärengraben
1962 , 12th century
1963 , Roman
1964 , Brenodor
1965 , 2
1966 , Zähringer
1967 , 12th century
1968 , Upper Burgundy
1969 , 1309
1970 ,

2509 , German
2510 , London
2511 , Soviet Relations
2512 , 
2513 , 
2514 , German
2515 , Soviets
2516 , 
2517 , 
2518 , Soviet
2519 , Jews
2520 , Jews
2521 , May
2522 , 
2523 , Carr
2524 , German
2525 , Carr
2526 , Viktor Suvorov
2527 , The
2528 , 1
2529 , Warsaw Pact
2530 , Warsaw Pact
2531 , 
2532 , Warsaw Pact
2533 , Warsaw Pact
2534 , Warsaw Pact
2535 , Warsaw Pact
2536 , Pact
2537 , Warsaw Pact
2538 , 1989
2539 , Polish
2540 , Warsaw Pact
2541 , Pact
2542 , Warsaw Treaty
2543 , Warsaw Pact
2544 , NATO
2545 , 
2546 , Western European
2547 , Warsaw Pact
2548 , USSR
2549 , German
2550 , 
2551 , NATO
2552 , USSR
2553 , British General
2554 , April 1954
2555 , Germany
2556 , NATO
2557 , Foreign Affairs
2558 , Czechoslovak President
2559 , Warsaw Pact
2560 , Army
2561 , 
2562 , Warsaw Pact
2563 , 1966
2564 , French Parliament
2565 , Warsaw Pact
2566 , 25
2567 , Hungarian
2568 , August 1968
2569 , Warsaw Pact
2570 , USSR
2571 , December 1989
2572 , 1
2573 , NATO
2574 , 1954
2575 , 12
257

...

6090 , Catholic Church
6091 , Pope
6092 , Pope
6093 , 
6094 , 
6095 , Pope
6096 , Rome
6097 , Ostia
6098 , 
6099 , 1630
6100 , 
6101 , Pope Urban
6102 , 
6103 , Roman Curia
6104 , English
6105 , English
6106 , Cardinal
6107 , Patriarchs
6108 , Patriarchs
6109 , Cardinals
6110 , 
6111 , 
6112 , 
6113 , Patriarch
6114 , 
6115 , Catholic Church
6116 , 
6117 , 
6118 , 
6119 , 
6120 , Pope
6121 , Pope
6122 , 
6123 , 
6124 , 
6125 , 
6126 , 
6127 , Cardinal Vicar
6128 , Ostia
6129 , 1962
6130 , Cardinal Vicar
6131 , 
6132 , Church
6133 , 
6134 , 
6135 , 
6136 , Lord
6137 , 
6138 , Lord
6139 , Catholic
6140 , 
6141 , 
6142 , 
6143 , 1460s
6144 , 
6145 , V
6146 , 
6147 , 1917
6148 , Code
6149 , 
6150 , 1962
6151 , 
6152 , 
6153 , 
6154 , Peter
6155 , Basilica
6156 , John Paul
6157 , 
6158 , 2005
6159 , 10
6160 , 1939
6161 , 1939
6162 , 
6163 , 
6164 , 
6165 , 
6166 , Dean
6167 , 
6168 , 
6169 , Catholic Church
6170 , 
6171 , 
6172 , 
6173 , 70
6174 , Pope Sixtus
6175 , College of
6176 , 45 %
6

6654 , the Guarda
6655 , the Guarda
6656 , 2001
6657 , 
6658 , 
6659 , 50 percent
6660 , 308
6661 , 308
6662 , 18
6663 , 
6664 , Navy
6665 , Navy
6666 , 
6667 , 
6668 , Army
6669 , Pandur II
6670 , Pandur II
6671 , M113 APC
6672 , 700
6673 , World War
6674 , 1974
6675 , Portuguese Empire
6676 , Forces
6677 , Pedro Passos
6678 , Portuguese
6679 , Portuguese
6680 , 1
6681 , 1974
6682 , Diário de
6683 , Notícias
6684 , 
6685 , 2007
6686 , Portuguese
6687 , BPN
6688 , President
6689 , Portuguese
6690 , Portuguese
6691 , Banks
6692 , Porto
6693 , World Travel
6694 , 1974
6695 , Portuguese Colonial
6696 , Portugal
6697 , 
6698 , European
6699 , Portuguese
6700 , 2011
6701 , 
6702 , May 2014
6703 , 
6704 , 
6705 , Grupo RAR
6706 , 
6707 , 
6708 , 
6709 , Combate
6710 , Portuguese
6711 , European
6712 , European
6713 , Portugal
6714 , 
6715 , Portuguese
6716 , 
6717 , Portuguese
6718 , Palmela
6719 , Covilhã
6720 , Portugal
6721 , 
6722 , Algarve
6723 , Algarve
6724 , Pombaline City
6725 , Por

7291 , 750
7292 , Mexico
7293 , San Diego
7294 , World War
7295 , Vizcaíno
7296 , Civic Center
7297 , Cabrillo
7298 , Census
7299 , 1850
7300 , 200
7301 , Sebastián Vizcaíno
7302 , 
7303 , Christian
7304 , Cabrillo
7305 , Kumeyaay
7306 , Junípero Serra
7307 , El Camino
7308 , Portolà
7309 , 1797
7310 , Presidio
7311 , Osuna
7312 , Mexican
7313 , 1838
7314 , Presidio Hill
7315 , Osuna
7316 , Stockton
7317 , Battle
7318 , Californios
7319 , San Diego
7320 , October 24
7321 , " Treaty
7322 , San Diego
7323 , Mexican
7324 , Mexican
7325 , Mexican
7326 , 1821
7327 , San Diego
7328 , San Diego
7329 , 1889
7330 , 1931
7331 , Old Town
7332 , San Diego
7333 , San Diego
7334 , the University
7335 , San Diego
7336 , World
7337 , 1915
7338 , Mexican
7339 , 20th century
7340 , World
7341 , Army
7342 , A
7343 , Air Station
7344 , U
7345 , Ryan Aeronautical
7346 , 1930
7347 , U
7348 , U
7349 , Japan
7350 , S
7351 , American
7352 , 1911
7353 , 1980s
7354 , Portuguese Azores
7355 , American
7356 , Rive

7909 , 000
7910 , Essonne
7911 , Essonne
7912 , 2011
7913 , 2011
7914 , 36
7915 , 
7916 , 7th arrondissement
7917 , Unemployment
7918 , Oise
7919 , 14 percent
7920 , 
7921 , 
7922 , 72
7923 , Notre Dame
7924 , 9
7925 , Marne
7926 , Paris
7927 , Collège
7928 , Pont de
7929 , Paris
7930 , 1817
7931 , 1817
7932 , 
7933 , Paris
7934 , Pierre Mignard
7935 , 1648
7936 , 1648
7937 , Romanticism
7938 , Les Demoiselles
7939 , 
7940 , 1905
7941 , 1825
7942 , Louis Daguerre
7943 , Trouvères
7944 , Robert Doisneau
7945 , late nineteenth
7946 , 2014
7947 , Mona Lisa
7948 , French Impressionist
7949 , Musée
7950 , Guimet Museum
7951 , National Museum
7952 , October 2014
7953 , Notre Dame
7954 , 14
7955 , January 2015
7956 , Opéra
7957 , Opéra Garnier
7958 , 1913
7959 , Théâtre des
7960 , 1680
7961 , 
7962 , 
7963 , 
7964 , 1889
7965 , Moulin Rouge
7966 , 1946
7967 , Grateful Dead
7968 , Epistolae ("
7969 , Victor Hugo
7970 , 1470
7971 , Middle Ages
7972 , Dame
7973 , Victor Hugo
7974 , Second Empire

8560 , Indian Rebellion
8561 , Edward Oxford
8562 , Indian Rebellion
8563 , 
8564 , Indian Rebellion
8565 , Indian Rebellion
8566 , Indian Rebellion
8567 , Indian Rebellion
8568 , 
8569 , 17
8570 , 
8571 , Brown
8572 , Victoria
8573 , 10
8574 , 
8575 , Queen Victoria
8576 , Leopold
8577 , Victoria
8578 , Prince Henry
8579 , 
8580 , Brown
8581 , Sir
8582 , Brown
8583 , Brown
8584 , 
8585 , Leopold
8586 , 
8587 , B
8588 , 50
8589 , 
8590 , B
8591 , Prince Edward
8592 , Leopold
8593 , Prince
8594 , B
8595 , 
8596 , 
8597 , Victoria
8598 , 50
8599 , 18
8600 , King George
8601 , 
8602 , Victoria
8603 , Kent
8604 , Victoria
8605 , 
8606 , Victoria
8607 , King George
8608 , 18
8609 , 
8610 , 
8611 , Duke of
8612 , 1820
8613 , 1830
8614 , Duke of
8615 , 1820
8616 , Duke of
8617 , 
8618 , 1830
8619 , Regency Act
8620 , William
8621 , Victoria
8622 , 
8623 , George IV
8624 , George IV
8625 , William
8626 , 18
8627 , William IV
8628 , 1830
8629 , Victoria
8630 , 
8631 , 18
8632 , 1830
8633 , 
863

In [128]:
num = 0
for i in test:
    for j in i["qa"]:
        num += 1
print (sum+0.0)/num

0.0710150064989
